In [ ]:
import re
import numpy as np
from itertools import combinations

In [ ]:
with open("input.txt", "r") as f:
# with open("example_input.txt", "r") as f:
    instructions = [line.strip() for line in f if line.strip()]

In [ ]:
def parse_lights(machine):
    lights_target = re.search(r'\[(.+?)\]', machine).group(1)
    return [1 if c == "#" else 0 for c in lights_target]    

In [ ]:
def parse_buttons(machine):
    return [tuple(map(int, match.split(','))) 
               for match in re.findall(r'\(([0-9,]+)\)', machine)]

In [ ]:
def parse_jolts(machine):
    return list(map(int, re.search(r'\{(.+?)\}', machine).group(1).split(',')))

In [ ]:
#part 1 confirmed
def min_button_presses(target, buttons):
    n = len(target)
    m = len(buttons)
    
    '''
    prep for matrix multiplication, the button is a row
    like buttons {(0,2)(1)} for a machine with 3 lights
    1 0 1 -> (0,2) 
    0 1 0 -> (1)
    '''
    A = [[0] * m for _ in range(n)]  # n x m matrix, all zeros
    for j, button_tuple in enumerate(buttons):
        for idx in button_tuple:
            # print(button_tuple, idx, j)
            A[idx][j] = 1

    for num_presses in range(m + 1):
        for combo in combinations(range(m), num_presses):
            lights = [0] * n
            for button_idx in combo:
                for i in range(n):
                    lights[i] ^= A[i][button_idx]  # XOR
            if lights == target:
                return num_presses

In [ ]:
''' note to self
# on part 1:
the boolean could be the numeric with the addition of a modulo 2
    XOR = +1%2

# on part 2:
if you're trying to solve for {1,2,1} with the batteries (0,2),(1)
by doing system of equations, it looks like
(columns are the batteries and rows are 1 when that battery has that number)
1x + 0y = 1  -> first row   = 0
0x + 1y = 2  -> second row  = 1
1x + 0y - 1  -> third row   = 2

and wit with matrices (remember dot product) A @ b = c
you would have 
|1 0|
|0 1| @ [x,y] = [1,2,1]
|1 0|

[x,y] = 1,2

and so our answer would be 1+2 = 3

However!!!!! there can be many [x,y] so we need to find the smallest combination of them (ergo LinearConstraint)

'''

' note to self\nthe boolean could be the numeric with the addition of a modulo 2\n    XOR = +1%2\n\nthis is like doing system of equations, \n1x + 0y = \nbut when working with matrices (remember dot product)\nA @ b = c\n\nyou would have \n1 0\n0 1 @ [x,y] \n1 0\n\nwhat is [x,y] so that c = [1,2,1]\n[x,y] = 1,2\n\nand so our answer would be 1+2 = 3\n\nnow LinearConstraint to optimize [x,y]\n\n'

In [ ]:
import numpy as np
from scipy.optimize import milp, LinearConstraint, Bounds

def min_button_presses_numeric(target, buttons):
    target = np.array(target)
    n = len(target)
    m = len(buttons)

    # Build matrix A: n x m
    A = np.zeros((n, m), dtype=int)
    for j, btn in enumerate(buttons):
        for idx in btn:
            if idx < n:
                A[idx, j] = 1

    #milp parameter building
    # Constraints: A @ x == target
    constraints = LinearConstraint(A, lb=target, ub=target)
    bounds = Bounds(lb=0, ub=np.inf)
    integrality = np.ones(m)  # 1 = integer

    result = milp(c=integrality, constraints=constraints, bounds=bounds, integrality=integrality)

    if result.success:
        x_opt = np.round(result.x).astype(int) #this is the array where every value is how many push for that index
        if np.array_equal(A @ x_opt, target):
            return int(x_opt.sum())

In [ ]:
total_light_presses = total_jolts_presses = 0
for machine in instructions:  
    # print(machine)  
    lights = parse_lights(machine)
    buttons = parse_buttons(machine)
    jolts = parse_jolts(machine)
    
    # part 1
    min_presses_lights = min_button_presses(lights, buttons)
    total_light_presses+=min_presses_lights

    # part 2
    min_presses_jolts = min_button_presses_numeric(jolts, buttons)
    total_jolts_presses+=min_presses_jolts
print("part 1:",total_light_presses)
print("part 2:",total_jolts_presses)
